## Import

In [1]:
!cp -r ../input/python-packages2 ./
!cp -r ../input/beng-pip-packages ./
!cp -r ../input/sctk-rover/SCTK ./

!tar xvfz ./beng-pip-packages/indic-nlp.tgz
!pip install ./indicnlp/indic_nlp_library-0.81-py3-none-any.whl -f ./ --no-index --find-links ./indicnlp/

!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps

!pip install datasets --no-index --find-links=file:///kaggle/input/hf-ds -U -q

cp: cannot stat '../input/sctk-rover/SCTK': No such file or directory
indicnlp/
indicnlp/packaging-23.1-py3-none-any.whl
indicnlp/pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
indicnlp/Babel-2.12.1-py3-none-any.whl
indicnlp/Pygments-2.16.1-py3-none-any.whl
indicnlp/MarkupSafe-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
indicnlp/Morfessor-2.0.6-py3-none-any.whl
indicnlp/requests-2.31.0-py3-none-any.whl
indicnlp/sphinxcontrib_jsmath-1.0.1-py2.py3-none-any.whl
indicnlp/urllib3-2.0.4-py3-none-any.whl
indicnlp/sphinx-6.2.1-py3-none-any.whl
indicnlp/six-1.16.0-py2.py3-none-any.whl
indicnlp/Jinja2-3.1.2-py3-none-any.whl
indicnlp/sphinx_rtd_theme-1.2.2-py2.py3-none-any.whl
indicnlp/sphinxcontrib_jquery-4.1-py2.py3-none-any.whl
indicnlp/imagesize-1.4.1-py2.py3-none-any.whl
indicnlp/idna-3.4-py3-none-any.whl
indicnlp/charset_normalizer-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
indicnlp/python_dateutil-2.8.2-py2.py3-none-any.wh

In [2]:
rm -r python-packages2 jiwer normalizer pyctcdecode pypikenlm beng-pip-packages indicnlp

In [3]:
import os
import sys
import shutil

import typing as tp
from pathlib import Path
import functools
import ctypes
from functools import partial
from dataclasses import dataclass

import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import json
import torch
import gc
import re
import abc
import math
import subprocess

import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoConfig, AutoModel, PretrainedConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import AutomaticSpeechRecognitionPipeline
from transformers import DataCollatorForTokenClassification
from transformers.pipelines.automatic_speech_recognition import *
from transformers.pipelines.pt_utils import KeyDataset
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.modelcard import ModelCard
from transformers.pipelines.base import ArgumentHandler, ChunkPipeline, infer_framework_load_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CTC_MAPPING_NAMES, MODEL_FOR_SPEECH_SEQ_2_SEQ_MAPPING_NAMES

from indicnlp.tokenize import indic_tokenize
from bnunicodenormalizer import Normalizer
from collections import defaultdict, Counter

import datasets
from datasets import Dataset
from datasets import Audio
import logging

logger = logging.getLogger(__name__)
from typing import (
    TYPE_CHECKING,
    Any,
    Collection,
    Dict,
    Iterable,
    List,
    Optional,
    Sequence,
    Tuple,
    TypeVar,
    Union,
)

import multiprocessing 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = DATA / "test_mp3s"
ensemble = False

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "wv-shru-v3-s6/"
DEFAULT_TRANSCRIPTION = "ও" 
kenlm_model_path = "/kaggle/input/lm-trained/lm_v9.binary"
ALPHA = 0.5
BETA = 1.0

decoder_kwargs={'beam_width':1024}

## Load audio paths

In [5]:
test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]
print(test.head())

             id                                           sentence
0  0f3dac00655e  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...
1  a9395e01ad21  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...
2  bf36ea8b718d  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...


## ASR model + Language model

In [6]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
token_list = list(sorted_vocab_dict.keys())

In [7]:
with open("/kaggle/input/lm-trained/vocab-4500000_v9.txt") as f:
    unigram_list = [t for t in f.read().strip().split("\n")]

In [8]:
decoder = pyctcdecode.build_ctcdecoder(
    token_list,
    kenlm_model_path,
    unigram_list,
    alpha = ALPHA,
    beta = BETA,
)

In [9]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")
print(device)

cuda


In [10]:
# Function to clean RAM & vRAM
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [11]:
pred_sentence_list = []
bnorm = Normalizer()

def normalize_sentence(sentence):
    if len(sentence)==0:
        return DEFAULT_TRANSCRIPTION
    _words = [bnorm(word)['normalized'] for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])    
    return sentence

In [12]:
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)
pipe = AutomaticSpeechRecognitionPipeline(
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    decoder=processor.decoder,
    device=0,
    decoder_kwargs=decoder_kwargs
)
pipe.decoder = processor.decoder
pipe.type = "ctc_with_lm"

with torch.no_grad():
    for audio_path in tqdm(test_audio_paths):
        w = librosa.load(audio_path, sr=16_000, mono=False)[0]
        w = np.trim_zeros(w, 'fb')

        text = pipe(w, chunk_length_s=14, stride_length_s=(6, 3))["text"]
        pred_sentence_list.append(text)

del pipe, processor, model, decoder
clean_memory()

  0%|          | 0/3 [00:00<?, ?it/s]

## Punctuation model

In [13]:
#Credits to Feedback 1 4th place solution: https://www.kaggle.com/competitions/feedback-prize-2021/discussion/313330
def create_ner_conditional_masks(id2label: Dict[int, str]) -> torch.Tensor:
    """Create a NER-conditional mask matrix which implies the relations between
    before-tag and after-tag.

    According to the rule of BIO-naming system, it is impossible that `I-Dog` cannot be
    appeard after `B-Dog` or `I-Dog` tags. This function creates the calculable
    relation-based conditional matrix to prevent from generating wrong tags.

    Args:
        id2label: A dictionary which maps class indices to their label names.

    Returns:
        A conditional mask tensor.
    """
    conditional_masks = torch.zeros(len(id2label), len(id2label))
    for i, before in id2label.items():
        for j, after in id2label.items():
            if after == "O" or after.startswith("B-") or after == f"I-{before[2:]}":
                conditional_masks[i, j] = 1.0
    return conditional_masks

def ner_beam_search_decode(
    log_probs: torch.Tensor, id2label: Dict[int, str], beam_size: int = 2
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Decode NER-tags from the predicted log-probabilities using beam-search.

    This function decodes the predictions using beam-search algorithm. Because all tags
    are predicted simultaneously while the tags have dependencies of their previous
    tags, the greedy algorithm cannot decode the tags properly. With beam-search, it is
    possible to prevent the below situation:

        >>> sorted = probs[t].sort(dim=-1)
        >>> print("\t".join([f"{id2label[i]} {p}" for p, i in zip()]))
        I-Dog 0.54  B-Cat 0.44  ...
        >>> sorted = probs[t + 1].sort(dim=-1)
        >>> print("\t".join([f"{id2label[i]} {p}" for p, i in zip()]))
        I-Cat 0.99  I-Dog 0.01  ...

    The above shows that if the locally-highest tags are selected, then `I-Dog, I-Dog`
    will be generated even the confidence of the second tag `I-Dog` is significantly
    lower than `I-Cat`. It is more natural that `B-Cat, I-Cat` is generated rather than
    `I-Dog, I-Dog`. The beam-search for NER-tagging task can solve this problem.

    Args:
        log_probs: The log-probabilities of the token predictions.
        id2label: A dictionary which maps class indices to their label names.
        beam_size: The number of candidates for each search step. Default is `2`.

    Returns:
        A tuple of beam-searched indices and their probability tensors.
    """
    # Create the log-probability mask for the invalid predictions.
    log_prob_masks = -10000.0 * (1 - create_ner_conditional_masks(id2label))
    log_prob_masks = log_prob_masks.to(log_probs.device)

    beam_search_shape = (log_probs.size(0), beam_size, log_probs.size(1))
    searched_tokens = log_probs.new_zeros(beam_search_shape, dtype=torch.long)
    searched_log_probs = log_probs.new_zeros(beam_search_shape)

    searched_scores = log_probs.new_zeros(log_probs.size(0), beam_size)
    searched_scores[:, 1:] = -10000.0

    for i in range(log_probs.size(1)):
        # Calculate the accumulated score (log-probabilities) with excluding invalid
        # next-tag predictions.
        scores = searched_scores.unsqueeze(2)
        scores = scores + log_probs[:, i, :].unsqueeze(1)
        scores = scores + (log_prob_masks[searched_tokens[:, :, i - 1]] if i > 0 else 0)

        # Select the top-k (beam-search size) predictions.
        best_scores, best_indices = scores.flatten(1).topk(beam_size)
        best_tokens = best_indices % scores.size(2)
        best_log_probs = log_probs[:, i, :].gather(dim=1, index=best_tokens)

        best_buckets = best_indices.div(scores.size(2), rounding_mode="floor")
        best_buckets = best_buckets.unsqueeze(2).expand(-1, -1, log_probs.size(1))

        # Gather the best buckets and their log-probabilities.
        searched_tokens = searched_tokens.gather(dim=1, index=best_buckets)
        searched_log_probs = searched_log_probs.gather(dim=1, index=best_buckets)

        # Update the predictions by inserting to the corresponding timestep.
        searched_scores = best_scores
        searched_tokens[:, :, i] = best_tokens
        searched_log_probs[:, :, i] = best_log_probs

    # Return the best beam-searched sequence and its probabilities.
    return searched_tokens[:, 0, :], searched_log_probs[:, 0, :].exp()

In [14]:
class PunctuationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        sentence = self.texts[item].split()

        tokenized_inputs = self.tokenizer(
            sentence, 
            truncation=True, 
            max_length=512, 
            padding='max_length', 
            is_split_into_words=True
        )
        word_ids = tokenized_inputs.word_ids()  

        tokenized_inputs = {key: torch.as_tensor(val) for key, val in tokenized_inputs.items()}

        word_ids2 = [w if w is not None else -1 for w in word_ids]
        tokenized_inputs['wids'] = torch.as_tensor(word_ids2)
        return tokenized_inputs

tag_values = ['O', 
        'B-END', 
        'I-END', 
        'B-COMMA', 
        'I-COMMA', 
        'B-QM', 
        'I-QM', 
        'B-EXCLM',
        'I-EXCLM',
        ]
labels_to_ids = {v:int(k) for k,v in enumerate(tag_values)}
ids_to_labels = {k:v for k,v in enumerate(tag_values)}

punctuation_dict = {
                            "O": " ", 
                            "B-QM": "? ",
                            "I-QM": "? ",
                            "B-COMMA": ", ", 
                            "I-COMMA": ", ", 
                            "B-END": "। ",
                            "I-END": "। ",
                            "B-EXCLM": "! ", 
                            "I-EXCLM": "! ", 
                            "B-HYPHEN":"-",
                            "I-HYPHEN":"-",
                            "B-DOT":".",
                            "I-DOT":".",
                        }

# LSTM Module credits to Feedback 1 3rd place solution: https://www.kaggle.com/competitions/feedback-prize-2021/discussion/313235
class ResidualLSTM(nn.Module):

    def __init__(self, d_model,rnn):
        super(ResidualLSTM, self).__init__()
        self.downsample=nn.Linear(d_model,d_model//2)
        if rnn=='GRU':
            self.LSTM=nn.GRU(d_model//2, d_model//2, num_layers=2, bidirectional=False, dropout=0.2)
        else:
            self.LSTM=nn.LSTM(d_model//2, d_model//2, num_layers=2, bidirectional=False, dropout=0.2)
        self.dropout1=nn.Dropout(0.2)
        self.norm1= nn.LayerNorm(d_model//2)
        self.linear1=nn.Linear(d_model//2, d_model*4)
        self.linear2=nn.Linear(d_model*4, d_model)
        self.dropout2=nn.Dropout(0.2)
        self.norm2= nn.LayerNorm(d_model)

    def forward(self, x):
        res=x
        x=self.downsample(x)
        x, _ = self.LSTM(x)
        x=self.dropout1(x)
        x=self.norm1(x)
        x=F.relu(self.linear1(x))
        x=self.linear2(x)
        x=self.dropout2(x)
        x=res+x
        return self.norm2(x)

class CustomModel(nn.Module):
    def __init__(self, config_path, num_labels):
        super(CustomModel, self).__init__()
        self.config = AutoConfig.from_pretrained(config_path)
        self.num_labels = num_labels

        self.model = AutoModel.from_pretrained(
            config_path, add_pooling_layer=False
        )

        self.lstm = ResidualLSTM(self.config.hidden_size,'LSTM')
        self.classification_head = nn.Linear(self.config.hidden_size,self.num_labels)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        x = self.lstm(sequence_output.permute(1,0,2)).permute(1,0,2)
        logits = self.classification_head(x)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [15]:
def postprocess(sentence):
    period_set = set(["?", "!", "।"])
    _words = [bnorm(word)['normalized'] for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    
    try:
        if sentence[-1] not in period_set:
            sentence+="।"
    except:
        sentence = DEFAULT_TRANSCRIPTION
    return sentence

In [16]:
pp_pred_sentence_list = [
    normalize_sentence(s) for s in tqdm(pred_sentence_list)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
pretrained_path = '/kaggle/input/indic-bert-v2'

tokenizer = AutoTokenizer.from_pretrained(pretrained_path)
model = CustomModel(pretrained_path, len(tag_values))

hyphen_caches = []
dot_caches = []
pp_pred_sentence_list_new = []
for sentence in pp_pred_sentence_list:
    hyphen_cache = []
    dot_cache = []
    sentence = sentence.replace("-", "- ").replace(".", ". ")
    sentence = sentence.split(" ")
    sentence = [w for w in sentence if len(w)>0]
    new_sentence = []
    for idx, s in enumerate(sentence):
        if s[-1]=="-":
            hyphen_cache.append(idx)
            new_sentence.append(s[:-1])
        elif s[-1]==".":
            dot_cache.append(idx)
            new_sentence.append(s[:-1])
        else:
            new_sentence.append(s)
    hyphen_caches.append(hyphen_cache)
    dot_caches.append(dot_cache)
    pp_pred_sentence_list_new.append(' '.join(new_sentence))

test_dataset = PunctuationDataset(pp_pred_sentence_list_new, tokenizer)

batch_size = 32
test_params = {'batch_size': batch_size,
                'shuffle': False,
                'num_workers': 2,
                'pin_memory':True
                }

test_dataloader = DataLoader(test_dataset, **test_params)

ensemble_preds = np.zeros((len(test_dataloader.dataset), 512, len(tag_values)), dtype=np.float32)
wids = np.full((len(test_dataloader.dataset), 512), -1)
all_tokens = []

model_ids = [
    "/kaggle/input/punct-3seed-ner/train_demo_new_v2plus_s200_ner",
    "/kaggle/input/punct-3seed-ner/train_demo_new_v2plus_s300_ner",
    "/kaggle/input/punct-3seed-ner/train_demo_new_v2plus_s400_ner",
]

for model_i, model_id in enumerate(model_ids):

    checkpoint = torch.load(f"{model_id}/pytorch_model.bin", map_location=device)
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()

    for batch_i, batch in tqdm(enumerate(test_dataloader)):
        # print(batch.keys())
        if model_i == 0: 
            wids[batch_i*batch_size:(batch_i+1)*batch_size,:batch['wids'].shape[1]] = batch['wids'].numpy()
            inputs = batch['input_ids'].numpy()
            for i in range(inputs.shape[0]):
                tokens_temp = tokenizer.convert_ids_to_tokens(inputs[i])
                tokens_temp = [t for t in tokens_temp if t !="[PAD]"]
                all_tokens.append(tokens_temp)
        # MOVE BATCH TO GPU AND INFER
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)

        with torch.no_grad():
            #with amp.autocast():
            outputs = model(ids, attention_mask=mask)
        all_preds = torch.nn.functional.softmax(outputs[0], dim=2).cpu().detach().numpy() 
        ensemble_preds[batch_i*batch_size:(batch_i+1)*batch_size,:all_preds.shape[1]] += all_preds
        
        # all_preds = torch.argmax(outputs[0], axis=-1).cpu().numpy() 
        del ids
        del mask
        del outputs
        del all_preds
        
    clean_memory()

ensemble_preds /= len(model_ids)
ensemble_preds = torch.from_numpy(ensemble_preds).log()
preds, pred_probs = ner_beam_search_decode(ensemble_preds, ids_to_labels, 4)
ensemble_preds = preds.cpu().numpy()

predictions = []
for text_i in range(ensemble_preds.shape[0]):
    prediction = []

    label_indices = ensemble_preds[text_i]
    word_ids = wids[text_i]
    tokens = all_tokens[text_i]
    new_tokens = []
    new_labels = []

    for i in range(1, len(tokens) - 1):
        if word_ids[i]!=word_ids[i-1]:
            if tokens[i].startswith("▁"):
                current_word = tokens[i][1:]
            else:
                current_word = tokens[i]

            new_labels.append(list(labels_to_ids.keys())[list(labels_to_ids.values()).index(label_indices[i])])
            for j in range(i + 1, len(tokens) - 1):
                if word_ids[j]==word_ids[j-1]:
                    current_word = current_word + tokens[j]
                if word_ids[j]!=word_ids[j-1]:
                    break
            new_tokens.append(current_word)
    full_text = ''
    tokenized_text = indic_tokenize.trivial_tokenize_indic(pp_pred_sentence_list_new[text_i])
        
    new_labels = ['blank' if x=='PAD' else x for x in new_labels] #fix for PAD predicted in outputs

    if len(tokenized_text) == len(new_labels):
        full_text_tokens = tokenized_text
    else:
        full_text_tokens = new_tokens

    hyphen_cache = hyphen_caches[text_i]
    dot_cache = dot_caches[text_i]
    for word_idx, (word, punctuation) in enumerate(zip(full_text_tokens, new_labels)):
        if punctuation=="O":
            if word_idx in hyphen_cache:
                punctuation = "B-HYPHEN"
            elif word_idx in dot_cache:
                punctuation = "B-DOT"
        full_text = full_text + word + punctuation_dict[punctuation]
    predictions.append(full_text.strip())
    
del model, ensemble_preds, test_dataloader, test_dataset, tokenizer, hyphen_caches, dot_caches
clean_memory()

Some weights of the model checkpoint at /kaggle/input/indic-bert-v2 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [18]:
pp_pred_sentence_list = [
    postprocess(s) for s in tqdm(predictions)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
test["sentence"] = pp_pred_sentence_list
test.to_csv("submission.csv", index=False)
print(test.head())

             id                                           sentence
0  0f3dac00655e                          একটু বয়স হলে একটি বিদেশি।
1  a9395e01ad21  কী কারণে তুমি এতাবৎ কাল পর্যন্ত এ দারুণ দৈব দু...
2  bf36ea8b718d  এ কারণে সরকার নির্ধারিত হারে পরিবহণজনিত ক্ষতি ...


## EOF